# *This notebook is work in progress and currently does not work*

In [ ]:
import numpy as np
import open3d as o3d
import tifffile as tf
from scipy.spatial import Voronoi, Delaunay
import pyvoro
from itertools import combinations
from collections import defaultdict
import trimesh
import matplotlib.pyplot as plt
%matplotlib qt

import os 
import bpy
import sys
from functools import reduce
import importlib
lib_dir = os.path.dirname(os.path.realpath('.'))
print(lib_dir)
if lib_dir not in sys.path:
  sys.path.append(lib_dir)

In [ ]:
import image_synthesis.utils as utils
import image_synthesis.rendering.triangulate as t
def reload():
  importlib.reload(utils)
  importlib.reload(t)
reload()

In [ ]:
width = 250
height = 250
depth = 250
cellCount = 10

## Use pyvoro to generate a 3d voronoi decomposition

In [ ]:
# points = np.array([[1.0, 2.0, 3.0], [4.0, 5.5, 6.0]])
points = np.array([[1.0, 2.0, 3.0], [4.0, 5.5, 6.0], [1, 5, 2], [9, 8, 7], [2, 9.5, 3]])
limits = np.array([[0.0, 10.0], [0.0, 10.0], [0.0, 10.0]])

v = pyvoro.compute_voronoi(
  points, # point positions
  limits, # limits
  2.0, # block size
  radii=[1.3, 1.4] # particle radii -- optional, and keyword-compatible arg.
)
v0_vertices = np.array(v[0]['vertices'])
v0_faces = [face['vertices'] for face in v[0]['faces']]
v

In [ ]:
def plot_points(points):
  ax = plt.figure().add_subplot(projection='3d')
  ax.scatter(points[:, 0], points[:, 1], points[:, 2], c='r')
  for i, point in enumerate(points):
    ax.text(point[0], point[1], point[2], str(i))
  plt.show()

In [ ]:
plot_points(np.array(v[0]['vertices']))

In [ ]:
def get_triangle_indices(points):
  assert points.shape[0] >= 3

  center = points.mean(axis=0)
  reference_vector = points[0] - center

  angles = [[utils.vector_angle(reference_vector, p - center), i] for i, p in enumerate(points)]
  angles.sort(key=lambda element: element[0])

  point_order = np.array(angles)[:, 1].flatten().astype(int)
  triangles = np.lib.stride_tricks.sliding_window_view(point_order, 3)

  if points.shape[0] > 3:
    triangles = np.append(triangles, [[point_order[-2], point_order[-1], point_order[0]]], axis=0)

  return triangles

### Plotting a polyhedral with open3d

In [ ]:
meshes = []
for face_vertices in v[0]['faces']:
  points = np.array(v[0]['vertices'])[face_vertices['vertices']]
  triangles = get_triangle_indices(points).copy()
  meshes.append(o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(points), o3d.utility.Vector3iVector(triangles)))

o3d.visualization.draw_geometries(meshes, mesh_show_back_face=True)

In [ ]:
voxel_grids = [o3d.geometry.VoxelGrid.create_from_triangle_mesh(mesh, voxel_size=0.05) for mesh in meshes]
o3d.visualization.draw_geometries(voxel_grids)

### Plotting a polyhedral with trimash

In [ ]:
points = np.empty((0, 3))
triangles = np.empty((0, 3))

for face_vertices in v[0]['faces']:
  surface_points = np.array(v[0]['vertices'])[face_vertices['vertices']]
  surface_triangles = get_triangle_indices(surface_points)
  
  surface_triangles = surface_triangles + points.shape[0]
  points = np.append(points, surface_points, axis=0)
  triangles = np.append(triangles, surface_triangles, axis=0)

mesh = trimesh.Trimesh(points, triangles)
mesh.show()

In [ ]:
points = np.empty((0, 3))
triangles = np.empty((0, 3))

for tetra in v:
  shape_points = np.array(tetra['vertices'])
  faces = [x['vertices'] for x in tetra['faces']]
  shape_triangles = t.triangulate_shape(shape_points, faces)
  triangles = np.append(triangles, shape_triangles + len(points), axis=0)
  points = np.append(points, shape_points, axis=0)

mesh = trimesh.Trimesh(points, triangles)

mesh.show()

## Use blender for scaling a polyhedron

In [ ]:
def get_scaled_shape_points(shape_idx: int, scale: float):
    in_vertices = v[shape_idx]['vertices']
    in_faces = [x['vertices'] for x in v[shape_idx]['faces']]

    mesh = bpy.data.meshes.new('poly')
    obj = bpy.data.objects.new('poly', mesh)

    col = bpy.data.collections.get('Collection')
    col.objects.link(obj)
    bpy.context.view_layer.objects.active = obj
    mesh.from_pydata(in_vertices, [], in_faces)

    bpy.ops.object.select_all(action='SELECT')

    bpy.ops.object.origin_set(type='ORIGIN_CENTER_OF_VOLUME', center='MEDIAN')
    obj.scale = (scale,) * 3
    bpy.ops.object.transform_apply(scale=True)

    bpy.ops.object.select_all(action='DESELECT')

    shape_vertices = np.empty((len(obj.data.vertices), 3))
    for i, vertex in enumerate(obj.data.vertices):
        shape_vertices[i] = [vertex.co[0], vertex.co[1], vertex.co[2]]
    return shape_vertices

In [ ]:
points = np.empty((0, 3))
triangles = np.empty((0, 3))

for i, tetra in enumerate(v):
  shape_points = get_scaled_shape_points(i, 0.9)
  faces = [x['vertices'] for x in tetra['faces']]
  shape_triangles = t.triangulate_shape(shape_points, faces)
  triangles = np.append(triangles, shape_triangles + len(points), axis=0)
  points = np.append(points, shape_points, axis=0)

mesh = trimesh.Trimesh(points, triangles)

mesh.show()

In [ ]:
voxel_grid = mesh.voxelized(0.05)
voxel_grid.fill()
voxels = np.array(voxel_grid.points) / 0.05

In [ ]:
voxels.min(axis=0)

In [ ]:
save_current_mesh()

### Is the mesh watertight?

In [ ]:
save()

In [ ]:
triangulate(v0_vertices[v0_faces[2]])

In [ ]:
def triangulate(points):
  tess = Delaunay(points)
  triangles = np.empty((0, 3), dtype=int)
  for tetra in tess.simplices:
    triangles = np.append(triangles, np.array(list((combinations(tetra, 3)))), axis=0)

In [ ]:
mesh = trimesh.Trimesh(points, triangles)
mesh.is_watertight

In [ ]:
pair_count = defaultdict(int)
def get_key(i, j):
  if f'{i},{j}' in pair_count:
    return f'{i},{j}'
  else:
    return f'{j},{i}'

for triangle in triangles:
  for i in range(3):
    key = get_key(triangle[i], triangle[(i+1) % 3])
    pair_count[key] += 1

In [ ]:
def save_current_mesh():
  voxel_grid = mesh.voxelized(0.05)
  voxel_grid.fill()
  voxels = np.array(voxel_grid.points) / 0.05
  save_voxel_grid.save_points_array('../../data/trimesh_polyhedral2.tiff', voxels)

In [ ]:
from image_synthesis.rendering import save_voxel_grid
importlib.reload(save_voxel_grid)

## Shrinking a polyhedron

In [ ]:
def get_shrinkable_faces(shape_idx: int):
  shrinkable_faces = []
  faces = [face['vertices'] for face in v[shape_idx]['faces']]

  for i, face in enumerate(faces):
    for j in range(3):
      if np.unique(v0_vertices[face][:, j]).shape[0] == 1:
        break
    else:
      shrinkable_faces.append(i)

  points_to_move = set()
  for shrinkable_face in shrinkable_faces:
    points_to_move = points_to_move.union(v0_faces[shrinkable_face])

  return shrinkable_faces

In [ ]:
def get_face_normals(shape_idx: int):
  faces = [face['vertices'] for face in v[shape_idx]['faces']]
  tri_faces = np.array([x[:3] for x in faces])

  def get_normal(points):
    return np.cross(points[0, :] - points[1, :], points[0, :] - points[2, :])

  return np.array(list(map(get_normal, v0_vertices[tri_faces])))

In [ ]:
def move_point(point, distance, shrinkable_faces, normals):
  # calculate the vector by which a point is moved
  faces_of_point = [i for i, face in enumerate(v0_faces) if point in face]
  shrinkable_faces_of_point = [i for i in faces_of_point if i in shrinkable_faces]

  if not shrinkable_faces_of_point:
    shrink_vector = np.array([0, 0, 0])
  else:
    shrink_vector = normals[shrinkable_faces_of_point].mean(axis=0)
    shrink_vector /= np.linalg.norm(shrink_vector)

  print('shrink vector:', shrink_vector)

  # move a point by it's shrink vector
  vertex = v0_vertices[point]
  moved_vertex = vertex + shrink_vector * distance

  # if the point is outside the limits, bring it back
  # first find coordinates of point that lie outside the boundaries
  moved_with_limits = np.concatenate((moved_vertex.reshape(-1, 1), limits[:, :1], limits[:, 1:2]), axis=1)
  outside_of_limits = np.logical_or(moved_with_limits[:, 0] < moved_with_limits[:, 1], moved_with_limits[:, 0] > moved_with_limits[:, 2])

  if not outside_of_limits.any():
    return moved_vertex

  # find out which boundary face is hit first by moving the point
  collision_faces = np.where(shrink_vector < 0, limits[:, 0], np.where(shrink_vector > 0, limits[:, 1], np.full(3, np.NaN)))
  dist_to_faces = (collision_faces - vertex) / shrink_vector

  # move the point to the first boundary hit
  factor_to_closest_face = np.min(dist_to_faces[~np.isnan(dist_to_faces)])
  vertex += shrink_vector * factor_to_closest_face

  # move the point by the projected normal s.t. it's on the plane
  proj_shrink_vector = np.where(outside_of_limits, np.zeros(3), shrink_vector)
  new_plane_d = np.dot(moved_vertex, shrink_vector)
  factor_to_plane = (new_plane_d - np.dot(shrink_vector, vertex)) / np.dot(proj_shrink_vector, shrink_vector)
  vertex += proj_shrink_vector * factor_to_plane

  return vertex

In [ ]:
vertices = np.array(v[0]['vertices'])
shrinkable_faces = get_shrinkable_faces(0)
normals = get_face_normals(0)
faces = np.array([2, 3, 5, 7, 6])

points = np.array([move_point(i, 0.1, shrinkable_faces, normals) for i in faces])
print(vertices[faces].__repr__())
points

In [ ]:
def shrink_shape(shape_idx: int, distance: float):
  vertices = np.array(v[shape_idx]['vertices'])
  shrinkable_faces = get_shrinkable_faces(shape_idx)
  normals = get_face_normals(shape_idx)

  points = np.array([move_point(i, distance, shrinkable_faces, normals) for i in range(len(vertices))])
  triangles = t.triangulate_shape(points, v0_faces)
  return points, triangles

### save it to an image

In [ ]:
points = np.empty((0, 3))
triangles = np.empty((0, 3))

for i in range(len(v)):
  shape_points, shape_triangles = shrink_shape(i, 0.1)
  shape_triangles += points.shape[0]
  points = np.append(points, shape_points, axis=1)
  triangles = np.append(triangles, shape_triangles, axis=1)

mesh = trimesh.Trimesh(points, triangles)
mesh.show()

In [ ]:
save_current_mesh()